<a href="https://www.kaggle.com/code/antonasnitsky/naya-ds2022-calassification-project?scriptVersionId=116606545" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/exercise-recognition/train.csv


# Import Libraries

In [2]:
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

Let's load the dataset

# EDA

In [3]:
df=pd.read_csv('/kaggle/input/exercise-recognition/train.csv')
df.head()

,pose_id,pose,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,...,z_left_heel,x_right_heel,y_right_heel,z_right_heel,x_left_foot_index,y_left_foot_index,z_left_foot_index,x_right_foot_index,y_right_foot_index,z_right_foot_index
0,0,squats_up,-0.382815,-48.231250,-54.405792,0.137189,-50.040543,-51.997875,0.502047,-50.058890,...,33.283375,-12.302963,52.377000,21.567219,16.609533,54.351162,15.890617,-16.779694,55.382572,2.050140
1,1,situp_down,54.146880,-12.822491,5.564175,56.762527,-11.221117,-0.363063,56.795986,-10.608183,...,-8.073100,-37.519974,4.045297,62.104233,-60.468560,6.281939,-8.185631,-52.589350,-3.852214,59.750070
2,2,situp_down,9.891440,-54.147266,85.344970,12.784414,-55.229970,88.534775,14.006874,-54.291880,...,-132.024460,-61.214890,53.040688,-16.786700,-5.844672,99.829030,-166.543060,-73.826830,38.470097,-40.519830
3,3,jumping_jacks_up,0.904673,-51.350130,-33.606970,1.338871,-53.172337,-30.013737,1.743913,-53.050697,...,51.615970,-15.543400,38.251835,34.033176,16.610210,46.572014,38.680614,-18.784650,45.098710,20.343110
4,4,jumping_jacks_down,-3.153129,-55.255062,-17.745928,-2.046205,-57.477790,-18.198952,-1.506304,-57.428230,...,-2.849554,2.811829,56.000500,56.605590,0.566345,66.231050,-19.252045,-3.451297,60.029484,45.503677


Since we have no need in POseId, let's get rid of it.

In [4]:
df.drop('pose_id', axis=1, inplace=True)
df.head()

,pose,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,z_left_eye,...,z_left_heel,x_right_heel,y_right_heel,z_right_heel,x_left_foot_index,y_left_foot_index,z_left_foot_index,x_right_foot_index,y_right_foot_index,z_right_foot_index
0,squats_up,-0.382815,-48.231250,-54.405792,0.137189,-50.040543,-51.997875,0.502047,-50.058890,-51.986694,...,33.283375,-12.302963,52.377000,21.567219,16.609533,54.351162,15.890617,-16.779694,55.382572,2.050140
1,situp_down,54.146880,-12.822491,5.564175,56.762527,-11.221117,-0.363063,56.795986,-10.608183,-0.378148,...,-8.073100,-37.519974,4.045297,62.104233,-60.468560,6.281939,-8.185631,-52.589350,-3.852214,59.750070
2,situp_down,9.891440,-54.147266,85.344970,12.784414,-55.229970,88.534775,14.006874,-54.291880,88.543910,...,-132.024460,-61.214890,53.040688,-16.786700,-5.844672,99.829030,-166.543060,-73.826830,38.470097,-40.519830
3,jumping_jacks_up,0.904673,-51.350130,-33.606970,1.338871,-53.172337,-30.013737,1.743913,-53.050697,-30.007776,...,51.615970,-15.543400,38.251835,34.033176,16.610210,46.572014,38.680614,-18.784650,45.098710,20.343110
4,jumping_jacks_down,-3.153129,-55.255062,-17.745928,-2.046205,-57.477790,-18.198952,-1.506304,-57.428230,-18.204160,...,-2.849554,2.811829,56.000500,56.605590,0.566345,66.231050,-19.252045,-3.451297,60.029484,45.503677


Let's  measure null values.

In [5]:
df.isnull().sum().sort_values(ascending=False)

pose                  0
z_right_index_1       0
x_right_hip           0
z_left_hip            0
y_left_hip            0
                     ..
z_mouth_left          0
y_mouth_left          0
x_mouth_left          0
z_right_ear           0
z_right_foot_index    0
Length: 100, dtype: int64

Let's see our labels

In [6]:
df['pose'].value_counts()

jumping_jacks_down    151
jumping_jacks_up      145
pullups_down          123
pushups_up            115
squats_up             111
pullups_up            108
squats_down           101
situp_down             82
pushups_down           82
situp_up               79
Name: pose, dtype: int64

# Split Dataset


In [7]:
df_train, df_test = train_test_split(df,test_size=0.2, random_state=2022)

# Transformations

Let's create more common label. We can clearly see that some poses are the same exercise' just in opposite directions.

In [8]:
df_train['exercise'] = np.nan

# If the pose contains the word 'squat' then the exercise is squats
df_train.loc[df_train['pose'].str.contains('squat'), 'exercise'] = 'squat'
df_train.loc[df_train['pose'].str.contains('jumping_jacks'), 'exercise'] = 'jumping jack'
df_train.loc[df_train['pose'].str.contains('push_up'), 'exercise'] = 'push up'
df_train.loc[df_train['pose'].str.contains('situp'), 'exercise'] = 'sit up'
df_train.loc[df_train['pose'].str.contains('pullups'), 'exercise'] = 'pull up'

df_train.head()

,pose,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,z_left_eye,...,x_right_heel,y_right_heel,z_right_heel,x_left_foot_index,y_left_foot_index,z_left_foot_index,x_right_foot_index,y_right_foot_index,z_right_foot_index,exercise
952,pullups_up,0.313043,-53.255184,34.796574,-0.309720,-55.588657,29.846674,-0.842028,-55.442886,29.844296,...,1.303038,58.831207,-2.338124,-3.286765,63.773090,0.241335,2.289710,63.730770,-4.813941,pull up
818,pullups_down,0.778168,-47.946900,14.272565,0.023860,-50.089700,10.790978,-0.400496,-50.105240,10.787390,...,5.641222,40.980568,37.263836,-5.306795,31.528297,43.436030,6.966193,38.683350,37.141500,pull up
623,jumping_jacks_down,-3.298060,-53.374184,-17.895926,-2.820273,-55.580120,-20.572166,-2.614269,-55.583797,-20.584206,...,8.021328,55.279730,46.703537,4.151109,63.919580,-15.517012,2.283572,58.787735,39.261047,jumping jack
835,pushups_up,47.115265,8.863159,-163.651380,49.035150,2.835183,-162.736860,49.582200,1.878313,-162.754580,...,-39.259735,-3.393133,175.623000,-27.682629,24.669918,214.855410,-36.639793,25.936058,160.655490,NaN
887,jumping_jacks_up,0.489088,-57.864536,-43.996494,1.666147,-59.480380,-39.555687,2.305762,-59.361313,-39.574875,...,-17.621288,53.900276,36.032560,18.648638,63.860780,20.753178,-20.558279,60.788000,11.201162,jumping jack


Now' we'll creat a new metric for every 3 coordinates of each point (vector mgnitude)

In [9]:
## Code for magnitude here